## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from itertools import islice
from IPython.core.display import display, HTML

from contract_utils import execute_bash, instantiate_contract, execute_contract, query_contract, get_ipfs_cid_from_str, get_proofs, ParseOutput, execute_contract_sdk, send_coins


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 1000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 10
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
DISPLAY_TX_EXECUTION = False
TEST_ACCOUNT_ADDRESS = 'bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0'
TEST_ACCOUNT_SEED = 'end spread blind steak install glare pride pony switch exit zone faint opinion march layer illness can nest fence top debate monitor supreme noble'

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(43)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in tqdm(range(NUMBER_OF_PARTICIPANTS)):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

  0%|          | 0/1000 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16492448010,QmWJabGXjQzQGBaufL4G52uWJ6pVXk6JAdSJ9FaP2er4HQ,0x9652c5b6b5fbb66725d04a9c60910bddeffea626,0xc8a81ae383de21ab9222b3a7c36d4851a481371e314b...,bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49,cosmos1kuqrfe0yaykad0dppea5ha79hd0paa8fnd4wtz,know welcome original south crane number blast...,bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49...,0xd4761ed5a2aaaf39b1129b488d5d674b395f8d43b494...,{'pub_key': 'Ak2zZhFmb277JrYppue8USwsUAxu7RTK9...,eyJwdWJfa2V5IjoiQWsyelpoRm1iMjc3SnJZcHB1ZThVU3...
1,1000000,john16492448011,QmVd9z3xpVuteLHfGxwyuqZ59RbqHJvo9rTnMmJydm2KFH,0xc054139731e52960f8074be8a5e95f3fcd721cf3,0xd569412597b83e8cc22d2d5afda4ca5c72e45e44f8a9...,bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj,cosmos1vd4dk5532ap70j3kcj8ren8mzey95cmrzp3uu4,swear jealous fix wait duck gossip amazing loy...,bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj...,0x08c64cbb5f56c4a68f310fcd76d0d9580546c74800de...,{'pub_key': 'Av+Omf/jShJAbKSzmeLoXMpfkCPOO23Ok...,eyJwdWJfa2V5IjoiQXYrT21mL2pTaEpBYktTem1lTG9YTX...
2,1000000,john16492448012,QmNP5sKxtM4CSGf7kTiWofVRnaM1BLypz1Bzp938CaELQe,0x28ad7d9904c8b3e114251b273e7d5ca5a875110c,0x201c6d3ad75b666634d4f6775719ee6bff005428d8d5...,bostrom19wakfjwwy2q2a4tt040zwlfje63hhgym8zc0vu,cosmos19wakfjwwy2q2a4tt040zwlfje63hhgymy3vujm,security ladder shop brush sauce edge category...,bostrom19wakfjwwy2q2a4tt040zwlfje63hhgym8zc0vu...,0x8b9f2830d4eb5142c14f0fd894a0f6b2ed0d912adae0...,{'pub_key': 'A50DR3BbqdA7GRRIe3p+whxcgy6Z7lYzU...,eyJwdWJfa2V5IjoiQTUwRFIzQmJxZEE3R1JSSWUzcCt3aH...
3,1000000,john16492448013,QmcgdRsZ9zhbwWwzk6YvjMPRPvAAPS1bHtCNyXcA8AWwNc,0x3bdac6c42a44eb2945089ce04c2ae76d421b8e32,0x7b34ba8f8da648fd458cb472e219cac67aeab9528c82...,bostrom17nvgh8e4628962yd08zdygpg77g06qvfvlwuav,cosmos17nvgh8e4628962yd08zdygpg77g06qvf0v60rt,horse file tissue valid tomorrow liar genius c...,bostrom17nvgh8e4628962yd08zdygpg77g06qvfvlwuav...,0xb03719a1aff6edb9a1d65d9873c32bec9ed9301a20fc...,{'pub_key': 'AhxaSBthlXKR9s6G5vWlAMotKdSR3ftR2...,eyJwdWJfa2V5IjoiQWh4YVNCdGhsWEtSOXM2RzV2V2xBTW...
4,1000000,john16492448014,QmZPLhT4qBiSHndYTLoW3uRyBWZ9RJqBUEceJDFju5FZKD,0xa90db3e1f3bf1f3db22c3eb395ab3357f79bc21a,0x232603ff11610671a4c9ab8a3268ffc13caba4eb714b...,bostrom19xfk4fypjdvh7klus3sqs6wjvv0frw8u7jyxe3,cosmos19xfk4fypjdvh7klus3sqs6wjvv0frw8uaps48k,spider mushroom page length point tenant plug ...,bostrom19xfk4fypjdvh7klus3sqs6wjvv0frw8u7jyxe3...,0x72b967a3bd50e7a7f3540caf4e91d3342ac578b7df06...,{'pub_key': 'AxUpfJBdo9SaxS06HZ/Ll8mkSsttxfhCe...,eyJwdWJfa2V5IjoiQXhVcGZKQmRvOVNheFMwNkhaL0xsOG...
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1000000,john1649244801995,QmdTu7ixrtsM6gCHony3L3YiMM3ZAyrerHNBF2BmH2c6sd,0xa796c56bf6351abb62536e8f1a71d10a8ba265cf,0xc176d2d0321c7b29c88e7dfaea7d8031b38ce09f5d36...,bostrom100n4qt5rd3jefhl5ezrcxlu4uf8hd34km5e850,cosmos100n4qt5rd3jefhl5ezrcxlu4uf8hd34kc8d52g,lumber chuckle scrap quantum clay ordinary dic...,bostrom100n4qt5rd3jefhl5ezrcxlu4uf8hd34km5e850...,0x6a2657437f37009f112e1774ab29f350a47b665c86a0...,{'pub_key': 'Ak72+BszxQHYtM+lE3skz9Qr+/yktGHWl...,eyJwdWJfa2V5IjoiQWs3MitCc3p4UUhZdE0rbEUzc2t6OV...
996,1000000,john1649244801996,QmbQfexR2AymywJtvPHNDXnv2sDzZPU2s7uNykigGY1NMZ,0x7d5c21ebcfaf008cfe7ba8228665d8478a47adee,0xd5d950b9d60dfcc801c98a8411434344423800e2243e...,bostrom1qewddun7a48s8985jux04d4w3xacum37mrcasf,cosmos1qewddun7a48s8985jux04d4w3xacum37csvwww,eye pair cry tent warfare abuse drop arena ste...,bostrom1qewddun7a48s8985jux04d4w3xacum37mrcasf...,0x8fca83c4c01fd63fa29d38206b4a2a0f421314185f54...,{'pub_key': 'AgR6I7cau7/qGeuQsUxfd0RiKoWYT3lxQ...,eyJwdWJfa2V5IjoiQWdSNkk3Y2F1Ny9xR2V1UXNVeGZkMF...
997,1000000,john1649244801997,Qmao6HPcDmMZz8uRgjPummgNbvud2KkNLAvgtZ9xW6sNry,0xa92d07ae7a4edc72bbba0eeb8bccdb816ad8ae4c,0xe5293951e6301d3e83fc3eb364f0e1e8dc1772705c82...,bostrom1rucwkkqr

### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
root_and_proofs_output, root_and_proofs_error = execute_bash(
    f'export NODE_OPTIONS=--max_old_space_size=4096 && '
    f'yarn start --input {ROOT_SOURCE_FILE_NAME} --output {PROOF_FILE_NAME}')
print(root_and_proofs_output)
with open(PROOF_FILE_NAME, 'r') as proof_file:
    root_and_proof_json = json.load(proof_file)
root = root_and_proof_json['merkle_root']
proofs_df = pd.DataFrame(root_and_proof_json['proofs'])
print(f'Merkle root: {root}')
print(f'Number of addresses in the Merkle tree: {len(proofs_df)}')

yarn run v1.22.17
$ ts-node index.ts --input root_testing_source_1000_addresses.json --output proof_testing_1000_addresses.json
Merkle root: b19dcb63a9fb28557e979d999eaa48dfe1e2ebd38f4f3dc8f472c52837887860
Number of addresses in the Merkle tree: 1999
Done in 3.64s.

Merkle root: b19dcb63a9fb28557e979d999eaa48dfe1e2ebd38f4f3dc8f472c52837887860
Number of addresses in the Merkle tree: 1999


In [5]:
NUMBER_OF_THREADS = 10
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
print(tasks)
with Pool(processes=NUMBER_OF_THREADS) as pool:
    res = pool.starmap(get_proofs, tasks)
assert res == [True] * len(res)

{('root_testing_source_1000_addresses.json', 'temp/proofs_0.json', 0, 1001)}
yarn run v1.22.17
$ ts-node index.ts --input root_testing_source_1000_addresses.json --output temp/proofs_0.json --start_index 0 --end_index 1001
Merkle root: b19dcb63a9fb28557e979d999eaa48dfe1e2ebd38f4f3dc8f472c52837887860
Number of addresses in the Merkle tree: 1001
Done in 2.79s.



In [6]:
roots = []
proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
for task in tasks:
    with open(task[1], 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    roots.append(root_and_proof_json['merkle_root'])
    proofs_df = proofs_df.append(pd.DataFrame(root_and_proof_json['proofs']), ignore_index=True)
assert roots == [roots[0]] * len(roots)
proofs_df

,address,amount,proof
0,0x9652c5b6b5fbb66725d04a9c60910bddeffea626,1000000,[452b463358c489b9e0189b016b310080b9e9fbcd6beff...
1,0xc054139731e52960f8074be8a5e95f3fcd721cf3,1000000,[6bb58e973986346c388792576715f282c70420036cefa...
2,0x28ad7d9904c8b3e114251b273e7d5ca5a875110c,1000000,[1c53cdfd53f1cb5e243607dc02b7821c3d2aa9d53d8c4...
3,0x3bdac6c42a44eb2945089ce04c2ae76d421b8e32,1000000,[9561ca1e1eb6e270b103585aa8c0072abe634f3fe8366...
4,0xa90db3e1f3bf1f3db22c3eb395ab3357f79bc21a,1000000,[c6733966b53fd8e52381b0ce9b7ecc5b2625aaee79fc4...
...,...,...,...
996,0x7d5c21ebcfaf008cfe7ba8228665d8478a47adee,1000000,[36d2307a53fe7b424779d2cb3009cb505c718aa56d8ad...
997,0xa92d07ae7a4edc72bbba0eeb8bccdb816ad8ae4c,1000000,[973c9930745e262272ceb092fb2785c7a007b12abd4e4...
998,0xd260b6e1ace04748313760719bdecc21fb82ea0b,1000000,[75565685ca4d8ee386ffb7eba95d1494b9533c4c6e947...
999,0x662ddb861fdbd085da590d74bf0d4e12c52b8dc5,1000000,[e73331ae9932fd23b621cf9bc834db5109aeba05e8aca...


In [7]:
cosmos_proofs_df = proofs_df[proofs_df.address.str.startswith('cosmos')][['address', 'proof']]
ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

claims_with_proofs_df = claims_df\
                .merge(
                    ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
                    how='left',
                    on='ethereum_address')\
                .merge(
                    cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
                    how='left',
                    on='cosmos_address')

claims_with_proofs_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16492448010,QmWJabGXjQzQGBaufL4G52uWJ6pVXk6JAdSJ9FaP2er4HQ,0x9652c5b6b5fbb66725d04a9c60910bddeffea626,0xc8a81ae383de21ab9222b3a7c36d4851a481371e314b...,bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49,cosmos1kuqrfe0yaykad0dppea5ha79hd0paa8fnd4wtz,know welcome original south crane number blast...,bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49...,0xd4761ed5a2aaaf39b1129b488d5d674b395f8d43b494...,{'pub_key': 'Ak2zZhFmb277JrYppue8USwsUAxu7RTK9...,eyJwdWJfa2V5IjoiQWsyelpoRm1iMjc3SnJZcHB1ZThVU3...,[452b463358c489b9e0189b016b310080b9e9fbcd6beff...,[b4ade17f02ffae186abf1aecba188bc31ebc4b8896f09...
1,1000000,john16492448011,QmVd9z3xpVuteLHfGxwyuqZ59RbqHJvo9rTnMmJydm2KFH,0xc054139731e52960f8074be8a5e95f3fcd721cf3,0xd569412597b83e8cc22d2d5afda4ca5c72e45e44f8a9...,bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj,cosmos1vd4dk5532ap70j3kcj8ren8mzey95cmrzp3uu4,swear jealous fix wait duck gossip amazing loy...,bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj...,0x08c64cbb5f56c4a68f310fcd76d0d9580546c74800de...,{'pub_key': 'Av+Omf/jShJAbKSzmeLoXMpfkCPOO23Ok...,eyJwdWJfa2V5IjoiQXYrT21mL2pTaEpBYktTem1lTG9YTX...,[6bb58e973986346c388792576715f282c70420036cefa...,NaN
2,1000000,john16492448012,QmNP5sKxtM4CSGf7kTiWofVRnaM1BLypz1Bzp938CaELQe,0x28ad7d9904c8b3e114251b273e7d5ca5a875110c,0x201c6d3ad75b666634d4f6775719ee6bff005428d8d5...,bostrom19wakfjwwy2q2a4tt040zwlfje63hhgym8zc0vu,cosmos19wakfjwwy2q2a4tt040zwlfje63hhgymy3vujm,security ladder shop brush sauce edge category...,bostrom19wakfjwwy2q2a4tt040zwlfje63hhgym8zc0vu...,0x8b9f2830d4eb5142c14f0fd894a0f6b2ed0d912adae0...,{'pub_key': 'A50DR3BbqdA7GRRIe3p+whxcgy6Z7lYzU...,eyJwdWJfa2V5IjoiQTUwRFIzQmJxZEE3R1JSSWUzcCt3aH...,[1c53cdfd53f1cb5e243607dc02b7821c3d2aa9d53d8c4...,NaN
3,1000000,john16492448013,QmcgdRsZ9zhbwWwzk6YvjMPRPvAAPS1bHtCNyXcA8AWwNc,0x3bdac6c42a44eb2945089ce04c2ae76d421b8e32,0x7b34ba8f8da648fd458cb472e219cac67aeab9528c82...,bostrom17nvgh8e4628962yd08zdygpg77g06qvfvlwuav,cosmos17nvgh8e4628962yd08zdygpg77g06qvf0v60rt,horse file tissue valid tomorrow liar genius c...,bostrom17nvgh8e4628962yd08zdygpg77g06qvfvlwuav...,0xb03719a1aff6edb9a1d65d9873c32bec9ed9301a20fc...,{'pub_key': 'AhxaSBthlXKR9s6G5vWlAMotKdSR3ftR2...,eyJwdWJfa2V5IjoiQWh4YVNCdGhsWEtSOXM2RzV2V2xBTW...,[9561ca1e1eb6e270b103585aa8c0072abe634f3fe8366...,NaN
4,1000000,john16492448014,QmZPLhT4qBiSHndYTLoW3uRyBWZ9RJqBUEceJDFju5FZKD,0xa90db3e1f3bf1f3db22c3eb395ab3357f79bc21a,0x232603ff11610671a4c9ab8a3268ffc13caba4eb714b...,bostrom19xfk4fypjdvh7klus3sqs6wjvv0frw8u7jyxe3,cosmos19xfk4fypjdvh7klus3sqs6wjvv0frw8uaps48k,spider mushroom page length point tenant plug ...,bostrom19xfk4fypjdvh7klus3sqs6wjvv0frw8u7jyxe3...,0x72b967a3bd50e7a7f3540caf4e91d3342ac578b7df06...,{'pub_key': 'AxUpfJBdo9SaxS06HZ/Ll8mkSsttxfhCe...,eyJwdWJfa2V5IjoiQXhVcGZKQmRvOVNheFMwNkhaL0xsOG...,[c6733966b53fd8e52381b0ce9b7ecc5b2625aaee79fc4...,NaN


## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [8]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [9]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8


### Set executor in the Subgraph Contracts

In [10]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                            contract_address=subgraph_contract_address,
                            gas=600000,
                            display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [11]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1g3pdg6xp89ehqlha8tshtsem4trgqcg4qd9qxfmc6u8g6plj7ntq32x2ey


### Initiate Class of Output Parsing

In [12]:
parse_output = ParseOutput(ipfs_client=ipfs_client,
                           address_dict= {
                               gift_contract_address: 'Gift Contract',
                               passport_contract_address: 'Passport Contract',
                               WALLET_ADDRESS: 'Passport Owner Address',
                               name_subgraph_contract_address: 'Name Subgraph Contract',
                               avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                               proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Register Merkle Root

In [13]:
root_register_output = execute_contract(execute_query=f'''{{"register_merkle_root":{{"merkle_root":"{root}"}}}}''',
                                        from_address=WALLET_ADDRESS,
                                        contract_address=gift_contract_address)
parse_output.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: b19dcb63a9fb28557e979d999eaa48dfe1e2ebd38f4f3dc8f472c52837887860
Gas used: 117,294
Tx hash: E3CB0B2B0008DB86095BE8E5C3A05FA9E277AF2326C3D0D47BD2CC6024697F0F


#### Get Merkle Root form the Gift Contract

In [14]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1g3pdg6xp89ehqlha8tshtsem4trgqcg4qd9qxfmc6u8g6plj7ntq32x2ey
{'data': {'merkle_root': 'b19dcb63a9fb28557e979d999eaa48dfe1e2ebd38f4f3dc8f472c52837887860'}}


### Send coins to new addresses

In [15]:
bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 1000
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in bostrom_addresses_chunks:
    send_output = send_coins(
            from_seed=WALLET_SEED,
            to_addresses=claims_with_proofs_df.bostrom_address.to_list(),
            amounts=[1] * len(claims_with_proofs_df.bostrom_address.to_list()),
            gas=min(71000 * len(claims_with_proofs_df.bostrom_address.to_list()), int(23e6)),
            display_data=DISPLAY_TX_EXECUTION)
    parse_output.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	amount: 1boot
	receiver: bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj
	amount: 1boot
	receiver: bostrom19wakfjwwy2q2a4tt040zwlfje63hhgym8zc0vu
	amount: 1boot
	receiver: bostrom17nvgh8e4628962yd08zdygpg77g06qvfvlwuav
	amount: 1boot
	receiver: bostrom19xfk4fypjdvh7klus3sqs6wjvv0frw8u7jyxe3
	amount: 1boot
	receiver: bostrom1gekafwaxa092d6x4k2fjay26wvc77un7pdyd7n
	amount: 1boot
	receiver: bostrom1da0yzdnz9rrj4zathm8cwvw6v25z37feysknja
	amount: 1boot
	receiver: bostrom1ce37c40jk9uz7akskup63x5v994f54nd8c5mgk
	amount: 1boot
	receiver: bostrom1dufp0qxvvauf8e4f2qr0gm47ggxazu3n36q28l
	amount: 1boot
	receiver: bostrom1d6ugkkfwrczktukdqaxj48w5kgdzzjfyvs70pg
	amount: 1boot
	receiver: bostrom1pkmdt3jga70afwx8u05quc0qkzuudr8lxm6asd
	amount: 1boot
	receiver: bostrom1pgvgd7lszh0mm4vzhrwud4h7sfsfzpgyz7lj6k
	amount: 1boot
	receiver: bostrom1jwtwne35zczy9wt0n4srw46yfe3ur5z7uzd0zk
	amount: 1boot
	receiver: bostrom1gh0yf9tng

### Create Passports

In [16]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"create_passport":{"avatar":claim_row["avatar"], "nickname":claim_row["nickname"]}},
        contract_address=passport_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=500000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        create_passport_json = create_passport(row, display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(create_passport_json, row=row)
    else:
        create_passport(row)

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Nickname -> Avatar
	Avatar -> Nickname
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49 wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 1
Gas used: 401,597
Tx hash: CB3467E0E041673A0B49CBB18D8DAD3E84321DD0FE17021CBE54B32A630B51A1


### Proof (Add) Address to Passports

In [17]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"proof_address": {"address":claim_row[network + "_address"], "nickname": claim_row["nickname"], "signature": claim_row[network + "_message_signature"]}},
        contract_address=passport_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=400000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        proof_ethereum_address_json = proof_address(row, display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(proof_ethereum_address_json, row=row)

        proof_cosmos_address_json = proof_address(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(proof_cosmos_address_json, row=row)
    else:
        proof_address(row)
        proof_address(row, network='cosmos')

0it [00:00, ?it/s]


Events

cyberlinks
	Nickname -> Ethereum Address
	Ethereum Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16492448011
	address: 0xc054139731e52960f8074be8a5e95f3fcd721cf3
Gas used: 263,051
Tx hash: 93FAC9D24252A5D9C22E2B611BB2B96BCED9775DDAD508CCE34D0E3EBB3BA9A0

Events

cyberlinks
	Nickname -> Cosmos Address
	Cosmos Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16492448011
	address: cosmos1vd4dk5532ap70j3kcj8ren8mzey95cmrzp3uu4
Gas

### Claim

In [18]:
def claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"claim": {"nickname": claim_row['nickname'], "gift_claiming_address": claim_row[network + "_address"], "gift_amount": str(claim_row['amount']), "proof": claim_row[network + "_proof"]}},
        contract_address=gift_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=400000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        claim_ethereum_json = claim(row, display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(claim_ethereum_json)

        claim_cosmos_json = claim(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(claim_cosmos_json)
    else:
        claim_ethereum_json_2 = claim(row)
        parse_output.parse_contract_execution_json(claim_ethereum_json_2)

        claim_cosmos_json_2 = claim(row, network='cosmos')
        parse_output.parse_contract_execution_json(claim_cosmos_json_2)

0it [00:00, ?it/s]


Events

coin received
	receiver: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x9652c5b6b5fbb66725d04a9c60910bddeffea626
	target: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	amount: 13000000
Gas used: 310,343
Tx hash: 5AE306D03DD911C82973226B687DDA5FF5FF397E9149F3FF7D4F7B19165CFF83

Events

coin received
	receiver: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer

### Release Gift

In [19]:
def release(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"release": {"gift_address": claim_row[network + "_address"]}},
        contract_address=gift_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=400000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        release_ethereum_json = release(row, display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(release_ethereum_json)

        release_cosmos_json = release(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(release_cosmos_json)
    else:
        release(row)
        release(row, network='cosmos')

0it [00:00, ?it/s]


Events

coin received
	receiver: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	sender: Gift Contract
	amount: 1290000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	gift_address: 0x9652c5b6b5fbb66725d04a9c60910bddeffea626
	stage: 9
	amount: 1290000
Gas used: 148,799
Tx hash: 0D0A73B6A255239B5A1310F54B25B178F2256F6F4C3311EFB176778245BFD2AB

Events

coin received
	receiver: bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1kuqrfe0yaykad0dppea5ha79hd0paa8fs7pa49 wasm /cosmwasm.wasm.v1.MsgExec

## Passport NFT testing
### Transfer Passport

In [20]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [23]:
def transfer_passport(claim_row: pd.Series, token_id: str,  to_address: str = '', display_data: bool = False):
    if to_address == '':
        to_address = claim_row['bostrom_address']
    return execute_contract_sdk(
        execute_msg={"transfer_nft": {"recipient": to_address, "token_id": str(token_id)}},
        contract_address=passport_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=500000,
        display_data=display_data)


for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        transfer_passport_json = transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
        break
    else:
        transfer_passport_json = transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS)

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom1vd4dk5532ap70j3kcj8ren8mzey95cmrpj90zj
	recipient: Passport Owner Address
	token_id: 2
Gas used: 359,234
Tx hash: A3D07D45E0FF6B9B3BFF9A152D4056AD392AF3DEDB3C3825F4148C5AF153311E


### Burn

In [24]:
def burn_passport(claim_row: pd.Series, token_id: str,  display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"burn": {"token_id": token_id}},
        contract_address=passport_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=400000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[2:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 2:
        burn_passport_json = burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(burn_passport_json, row=row)
        break
    else:
        burn_passport_json_2 = burn_passport(row, token_id=get_passport_id(row['bostrom_address']))

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom19wakfjwwy2q2a4tt040zwlfje63hhgym8zc0vu wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom19wakfjwwy2q2a4tt040zwlfje63hhgym8zc0vu
	token_id: 3
Gas used: 360,228
Tx hash: FAC929AE542165D6ABEEE76841A0D15E2BC67935B5257B9AA25D79583D3CBC34


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
